In [1]:
# 判断是否有高血压
### 如果自己报告有 或者 血压测量有  均认为有高血压
# SEQN    编码
# BPQ020  自我报告高血压 1有； 2没有； NAN缺失
# BPXSY1  BPXSY2  BPXSY3  三次收缩压
# BPXDI1  BPXDI2  BPXDI3  三次舒张压
# systolic blood	   收缩压平均值
# diastoloc blood	   舒张压平均值
# hypertension_level  是否有高血压 1有 0没有 NA 缺失

import pandas as pd
import numpy as np

In [2]:
# 获取自我报告是否高血压的数据 - blood_pressure_extract_df
blood_pressure_questionnaire_df = pd.read_sas('E:/work/Paper/NHANES/0910Analysis Data/Blood Pressure/BPQ_F.XPT')

blood_pressure_extract_features = [
    'SEQN',             
    'BPQ020',           #是否有人告知高血压
]
blood_pressure_extract_df = blood_pressure_questionnaire_df[blood_pressure_extract_features]
blood_pressure_extract_df['BPQ020'].replace([7,9], np.nan, inplace=True)

C:\Users\Gstar\AppData\Local\Temp\ipykernel_8992\1947466226.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blood_pressure_extract_df['BPQ020'].replace([7,9], np.nan, inplace=True)


In [3]:
# 获取测量血压的数据 - blood_pressure_examination_extract_df
blood_pressure_examination_df = pd.read_sas('E:/work/Paper/NHANES/0910Analysis Data/Blood Pressure/BPX_F.XPT')
blood_pressure_examination_extract_features = [
    'SEQN',     #编码
    'BPXSY1',   #收1
    'BPXSY2',   #收2
    'BPXSY3',   #收3
    'BPXDI1',   #舒1
    'BPXDI2',   #舒2
    'BPXDI3',   #舒3
]
blood_pressure_examination_extract_df = blood_pressure_examination_df[blood_pressure_examination_extract_features]
blood_pressure_examination_extract_df['systolic blood'] = blood_pressure_examination_extract_df.loc[:,['BPXSY1','BPXSY2','BPXSY3']].mean(axis=1)    #计算平均收缩压
blood_pressure_examination_extract_df['diastoloc blood'] = blood_pressure_examination_extract_df.loc[:,['BPXDI1','BPXDI2','BPXDI3']].mean(axis=1)   #计算平均舒张压

C:\Users\Gstar\AppData\Local\Temp\ipykernel_8992\1033929468.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blood_pressure_examination_extract_df['systolic blood'] = blood_pressure_examination_extract_df.loc[:,['BPXSY1','BPXSY2','BPXSY3']].mean(axis=1)    #计算平均收缩压
C:\Users\Gstar\AppData\Local\Temp\ipykernel_8992\1033929468.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blood_pressure_examination_extract_df['diastoloc blood'] = blood_pressure_examination_extract_df.loc[:,['BPXDI1','BPXDI2','BPXDI3

In [4]:
# 将两者数据合并 - hypertension_df
hypertension_df = pd.merge(left=blood_pressure_extract_df, right=blood_pressure_examination_extract_df, on='SEQN', how='outer')

In [5]:
# 判断是否高血压
def judge_hypertension(x):
    if x['BPQ020'] == 1:
        judge = 1
    elif pd.isna(x['BPQ020']):
        if pd.isna(x['systolic blood']) | (pd.isna(x['diastoloc blood'])):
            judge = np.nan
        else:
            if (x['systolic blood'] > 140) | (x['diastoloc blood'] > 90):
                judge = 1
            else:
                judge = 0
    else:
        if (x['systolic blood'] > 140) | (x['diastoloc blood'] > 90):
            judge = 1
        else:
            judge = 0
    return judge

hypertension_df['hypertension_level'] = hypertension_df.apply(judge_hypertension, axis=1)

In [6]:
hypertension_df.to_excel('E:/work/Paper/NHANES/0910Analysis Data/Final data/blood_pressure.xlsx')